# 高性能Pandas：eval()与query()

Python 数据科学生态环境的强大力量建立在 NumPy 与 Pandas 的基础之上，并通过直观的语法将基本操作转换成 C 语言：在 NumPy 里是向量化 / 广播运算，在 Pandas 里是分组型的运算。

> 所谓抽象，就是用户只需要知道一个事物能提供什么功能就行了，不必知道里面的工作原理。或者也叫不用知道实现方式。
>
> 抽象，简单地说就是，事物的原理是透明的。

## query()和eval()的设计动机：复合代数式

np和pd都支持向量化运算，比如你可以对数组求和：

In [2]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(int(1E6))
y = rng.rand(int(1E6))
%timeit x+y

6.24 ms ± 540 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


这种运算在处理复合代数式问题中的效率比较低，例如下面的表达式：

In [3]:
mask = (x > 0.5) & (y < 0.5)

相当于：

In [4]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

Numexpr程序库简单地说就是用了NumPy风格的字符串代数式进行运算：

In [5]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

不知道作者在说什么。

## 用pandas.eval()实现高性能运算

pandas的eval()函数用字符串代数式实现了DataFrame的高性能运算。例如下面的DataFrame：

In [6]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))


F:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
F:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


普通的Pandas方法计算4个df的和：

In [7]:
%timeit df1+df2+df3+df4

154 ms ± 12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


pd.eval()方法：

In [8]:
%timeit pd.eval('df1+df2+df3+df4')

69.8 ms ± 5.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


快一倍左右，并且内存消耗少。

In [9]:
np.allclose(df1+df2+df3+df4, pd.eval('df1+df2+df3+df4'))

True

In [11]:
np.allclose?

### pd.eval()支持的运算

支持多运算。

In [12]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) for i in range(5))

#### 算术运算符

支持所有的运算符，例如：

In [13]:
result1 = -df1 * df2 / (df3+df4) - df5
result2 = pd.eval('-df1 * df2 / (df3+df4) - df5')
np.allclose(result1, result2)

True

#### 比较运算符

包括链式代数式(chained expression)：

In [15]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('(df1 < df2) & (df2 <= df3) & (df3 != df4)')
np.allclose(result1, result2)

True

#### 位运算符

支持&和|等位运算符。

In [16]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

还可以使用and和or。

In [17]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result3, result1)

True

#### 对象属性与索引

使用obj.attr获得对象属性，使用obj\[index\]获得对象索引。

In [18]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### 其他运算

pd.eval()不支持函数调用，条件语句，循环和更复杂的运算。可以使用Numexpr实现。

## 用DataFrame.eval()实现列间运算

pd.eval()是pandas的顶层函数，所以DataFrame有一个eval()方法可以做类似的运算。借助列名进行运算。

In [19]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


可以用前面的方法计算3列：

In [21]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

In [22]:
result1.head()

0   -0.841283
1   -0.360327
2   -3.197758
3   -1.642291
4   -1.903118
dtype: float64

通过DataFrame.eval()可以用列名实现简洁的代数式：

In [23]:
result3 = df.eval('(A+B) / (C-1)')
np.allclose(result1, result3)

True

### 用DataFrame.eval()新增列

DataFrame.eval()还可以创建新的列。演示：

In [24]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


新增D列：

In [26]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


修改已有的列：

In [27]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


### DataFrame.eval()使用局部变量

支持使用@符号使用python的局部变量，如下所示：

In [28]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

为什么会出现这种用法，不能直接使用吗？

In [31]:
df.mean(1).head()

0    0.100740
1   -0.154913
2    0.534579
3   -0.036899
4    0.500769
dtype: float64

@符号表示这是一个变量名称，而不是一个列名称。

2个命名空间：列名称空间，变量名称空间。

@符号只能在DataFrame.eval()方法中使用，而不能在pandas.eval()中使用。后者只能获取一个命名空间的内容。

### DataFrame.query()方法

这个方法有什么用呢？

In [32]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

这个也是用DataFrame列创建的代数式，但是不能用DataFrame.eval()，因为里面包含了全部列？？？

In [33]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

也支持@符号的使用。

In [34]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

### 性能决定使用时机

涉及NumPy数组或者Pandas的DataFrame的复合代数式都会产生临时数组，例如：

In [36]:
x = df[(df.A < 0.5) & (df.B < 0.5)]
x.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
7,0.406639,0.128631,0.160742,1.729526
8,0.020236,0.354904,0.067919,-4.927445
16,0.110796,0.100477,0.561988,0.018362


基本等价于：

In [38]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]
x.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
7,0.406639,0.128631,0.160742,1.729526
8,0.020236,0.354904,0.067919,-4.927445
16,0.110796,0.100477,0.561988,0.018362


如果临时DataFrame的内存比你的系统内存还要大，那么要考虑使用eval()和query()代数式了。

预估变量的内存消耗：

In [42]:
df.values.nbytes

32000

即使不占用所有的内存，eval和query也节省内存和快速。

相关细节可以看 Pandas 文 档 中“Enhancing Performance”。